## Importando Librerias

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import datetime
import re
import pickle as pk
import pandas as pd
import math

In [2]:
df_marcas = pd.read_excel('Requerimientos/Homologacion_Marcas.xlsx')
marcas = df_marcas.values.tolist()

In [3]:
#Tiempo-Fecha
tiempo = datetime.datetime.now()
fecha_carpeta = tiempo.strftime('Chileautos_%d_%m_%Y')
fecha_documento = tiempo.strftime("%d-%m-%Y_%H-%M")
fecha_dato = tiempo.strftime("%Y-%m-%d %H:%M:%S")

#Directorios
dire = os.getcwd()
if os.path.isdir(dire + '\\Datos'):
    os.chdir(dire + '\\Datos')
else:
    os.mkdir('Datos')
    os.chdir(dire + '\\Datos')

dire = os.getcwd()
if os.path.isdir(dire + '\\' + fecha_carpeta):
    os.chdir(dire + '\\' + fecha_carpeta)
else:
    os.mkdir(fecha_carpeta)
    os.chdir(dire + '\\' + fecha_carpeta)

In [4]:
#Lista Contenedora de datos
data = []

#Nombre archivo
nombre_archivo = 'Chileautos_scrape_' + str(fecha_documento)

## Constantes, Iteradores, URL´s base y Listas

In [5]:
#Constantes
anio_base = 2000
anio_actual = int(datetime.date.today().strftime('%Y'))
max_anuncios = 1008
kms = [1000, 20000, 30000, 40000, 50000, 60000, 65000, 70000, 80000, 90000, 95000, 100000, 120000, 150000, 200000, 300000, 500000, 1000000]
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0'}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

#URL´s base
URLs_base_anio = []
for anio in range(anio_base, anio_actual +1):
    URL_anio = 'https://www.chileautos.cl/vehiculos/?q=(And.Servicio.chileautos._.TipoVeh%c3%adculo.Autos._.Tipo.Usado._.Kilometraje.range({}..)._.Precio.range(100000..99999999)._.Ano.range({}..{}).)&sort=lastupdated'.format(kms[0],anio,anio)
    page = requests.get(URL_anio, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    n_anuncios = int(re.sub("[^0-9]", "", soup.find('h1').contents[0].split()[0]))
    too_big = False
    if n_anuncios > max_anuncios:
        too_big = True
    URLs_base_anio.append([anio, n_anuncios, URL_anio, too_big])

URLs_base_anio_ready = []
anio_not_ready = []
for anio in URLs_base_anio:
    if not anio[3]:
        URLs_base_anio_ready.append([anio[0], anio[1], anio[2]])
    else:
        anio_not_ready.append([anio[0], anio[1]])

In [6]:
URLs_base_anio_not_ready = []
for anio in anio_not_ready:
    ready = False
    start_ite_kms = 0
    end_ite_kms = 1
    t_anuncios = 0
    while not ready:
        URL_anio = 'https://www.chileautos.cl/vehiculos/?q=(And.Servicio.chileautos._.TipoVeh%c3%adculo.Autos._.Tipo.Usado._.Kilometraje.range({}..{})._.Precio.range(100000..99999999)._.Ano.range({}..{}).)&sort=lastupdated'.format(kms[start_ite_kms],kms[end_ite_kms],anio[0],anio[0])
        page = requests.get(URL_anio, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        n_anuncios = int(re.sub("[^0-9]", "", soup.find('h1').contents[0].split()[0]))
        if n_anuncios < max_anuncios-240 and t_anuncios+n_anuncios < anio[1]:
            end_ite_kms += 1
        else:
            URLs_base_anio_not_ready.append([anio[0], n_anuncios, URL_anio])
            t_anuncios += n_anuncios
            if(t_anuncios < anio[1]):   
                start_ite_kms = end_ite_kms
                end_ite_kms += 1
            else:
                ready = True
        

In [7]:
URLs = URLs_base_anio_ready + URLs_base_anio_not_ready

In [8]:
URLs[0]

[2000,
 55,
 'https://www.chileautos.cl/vehiculos/?q=(And.Servicio.chileautos._.TipoVeh%c3%adculo.Autos._.Tipo.Usado._.Kilometraje.range(1000..)._.Precio.range(100000..99999999)._.Ano.range(2000..2000).)&sort=lastupdated']

In [9]:
for URL_ite in URLs:
    #Iniciar parametros
    contador = int(URL_ite[1]/12)*12

    #Loop que itera desde la pagina 84 hasta la pagina 1
    while contador >= 0:

        #Ajuste de URL para la pagina i
        contador_URL = str(contador)
        URL = URL_ite[2] + '&offset=' + contador_URL

        #Descarga y parsing del contenido de la pagina i
        page = requests.get(URL, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')

        #Levantamiento de los avisos disponibles en la pagina i
        avisos = soup.find_all('div', class_='listing-item')

        #Este loop itera sobre todos los avisos contenidos en la paguina i
        for aviso in reversed(avisos):

            #Seleccion y Elaboracion de datos
            #Enlace y Año
            link = '/vehiculos/detalles/x/' + aviso.get('data-webm-networkid')
            ano = aviso.find_all('h3')[0].contents[0].contents[0][0:4]

            #Tipo de publicante
            if(aviso.get('data-webm-vehcategory') == 'Agencia'):
                tipo_publicacion = 'DEALER'
            elif(aviso.get('data-webm-vehcategory') == 'Particular'):
                tipo_publicacion = 'PRIVATE'


            #Detalles: Kilometraje, Transmision y Combustible

            #palabras a detectar para la clasificicacion de la informacion
            detecta_km = 'km'
            detecta_transmision = ['Automática', 'Manual']
            detecta_combustible = ['Bencina', 'Diesel']

            #Levantamiento de detalles para el aviso i
            detalles = aviso.find_all('div', class_='key-detail-value')

            #Iterador de detalles para el aviso i
            n_detalle = 0
            #Variables contenedoras de la informacion
            data_km = ''
            data_transmision = ''
            data_combustible = ''

            #Este loop itera sobre los detalles del aviso i, detecta el tipo de informacion y la almacena
            for detalle in detalles:
                #Transforma  el detalle a un string para analizar el contenido
                detecta_str = str(detalle.contents)

                #Este if detecta si la informacion es sobre el Kilometraje
                if detecta_km in detecta_str:
                    #Limpia el string
                    detecta_str.rstrip('\r\n')
                    #Remueve simbolos no numericos
                    km_i = re.sub("[^0-9]", "", detecta_str)
                    #Transforma el dato a numero
                    km = int(km_i)
                    #Guarda la informacion
                    data_km = km
                    #Actualiza el iterador de detalles
                    n_detalle += 1
                #Este elif detecta si no hay informacion sobreel Kilometraje y si es que si hay sobre la Transmision y la codifica    
                elif(n_detalle == 0):
                    #En caso de que no exista informacion sobre Kilometraje insrta un guion
                    data_km = '-'
                    #Detecion y guardado de la informacion sobre la Transmision
                    if detecta_transmision[0] in detecta_str:
                        data_transmision = 'AUT'
                    elif detecta_transmision[1] in detecta_str:
                        data_transmision = 'MEC'    
                    #Se actualiza el iterador
                    n_detalle += 2
                #Este elif detecta si es que hay informacion sobre la Transmision y/o el Combustible y la codifica    
                elif(n_detalle > 0):
                    #Detecion y guardado de la informacion sobre la Transmision
                    if detecta_transmision[0] in detecta_str:
                        data_transmision = 'AUT'
                    elif detecta_transmision[1] in detecta_str:
                        data_transmision = 'MEC'
                    #Detecion y guardado de la informacion sobre el Combustible
                    elif detecta_combustible[0] in detecta_str:
                        data_combustible = 'BENCINA'
                    elif detecta_combustible[1] in detecta_str:
                        data_combustible = 'DIESEL'
                    #Se actualiza el iterador
                    n_detalle += 1


            #Aqui se Homologara el nombre de marca en chileautos con el nombre de la marca de autotraders
            buscar_marca = True
            marca_aviso = ''
            for marca in marcas:
                if str(aviso.get('data-webm-make')) == marca[0]:
                    marca_aviso = marca[1]
                    buscar_marca = False
            if(buscar_marca):
                marca_aviso = str(aviso.get('data-webm-make')).upper()
                buscar_marca = False

            #Se elaboran, formatean y pasan a variables los datos restantes
            id_chileautos = aviso.get('data-webm-networkid')
            modelo = str(aviso.get('data-webm-model')).upper()
            precio = int(aviso.get('data-webm-price'))
            region = str(aviso.get('data-webm-state')).upper()
            titulo = str(aviso.find_all('h3')[0].contents[0].contents[0][:]).upper()

            #Se genera la lista con la informacion del anuncio i
            dato = [id_chileautos, tipo_publicacion, ano, marca_aviso, modelo, titulo, data_km, precio, data_transmision, link, fecha_dato, region, data_combustible]

            #Se Añade la lista con la informacion del anuncio i la lista con los datos de todos los anuncios
            data.append(dato)    

        #Actualizacion de los iteradores
        contador += -12

In [10]:
#Abriendo o creando el archivo
archivo = open(nombre_archivo, 'wb')

#Pickling
pk.dump(data, archivo)

#Cerrando Archivo
archivo.close()

In [11]:
data[0]

['CL-AD-11511550',
 'PRIVATE',
 '2000',
 'NISSAN',
 'PATHFINDER',
 '2000 NISSAN PATHFINDER 3.3',
 277000,
 5200000,
 'AUT',
 '/vehiculos/detalles/x/CL-AD-11511550',
 '2021-07-01 13:46:39',
 'METROPOLITANA DE SANTIAGO',
 'BENCINA']

In [12]:
print(len(data))
print(data[0])

4272
['CL-AD-11511550', 'PRIVATE', '2000', 'NISSAN', 'PATHFINDER', '2000 NISSAN PATHFINDER 3.3', 277000, 5200000, 'AUT', '/vehiculos/detalles/x/CL-AD-11511550', '2021-07-01 13:46:39', 'METROPOLITANA DE SANTIAGO', 'BENCINA']


In [13]:
nombre_archivo

'Chileautos_scrape_01-07-2021_13-46'